# Specialization

To be fast, Julia needs to **specialize** code, that is compile specific native versions of the code utilizing the type information. **The better the specialization the faster the code!**

## "Just ahead of time" compilation

* Julia **specializes on the types of function arguments** and 
* compiles efficient machine code **when a function is called for the first time** (with these input argument types).

If the same function is called again with the same input argument types, the already existing machine code is reused.


In [27]:
func(x,y) = 2x + y

func (generic function with 1 method)

In [28]:
x = [1.2, 3.4, 5.6] # Vector{Float64}
y = [0.4, 0.7, 0.9] # Vector{Float64}

@time func(x,y);
@time func(x,y);

  0.005169 seconds (441 allocations: 30.921 KiB, 99.60% compilation time)
  0.000007 seconds (2 allocations: 160 bytes)


**First call:** compilation + running the code

**Second call:** running the code


In [29]:
@time func(x,y);

  0.000008 seconds (2 allocations: 160 bytes)


If one of the input types changes, Julia compiles a new specialization of the function!


In [30]:
typeof(x)

Vector{Float64} (alias for Array{Float64, 1})

In [31]:
x = [1, 3, 5]

3-element Vector{Int64}:
 1
 3
 5

In [32]:
typeof(x)

Vector{Int64} (alias for Array{Int64, 1})

In [33]:
@time func(x,y); # Vector{Int64}, Vector{Float64}
@time func(x,y);

  0.132855 seconds (166.58 k allocations: 11.230 MiB, 99.98% compilation time)
  0.000010 seconds (2 allocations: 160 bytes)


We now have two efficient native codes in the cache: one for all `Vector{Float64}` inputs and another one for `Vector{Int64}` as the first and `Vector{Float64}` as the second argument type.

In [34]:
methods(func)

# 1 method for generic function "func" from Main:
 [1] func(x, y)
     @ ~/repos/JuliaWorkshops/JuliaHLRS23/notebooks/1_2_specialization.ipynb:1

In [35]:
using MethodAnalysis
methodinstances(func)

2-element Vector{Core.MethodInstance}:
 MethodInstance for func(::Vector{Float64}, ::Vector{Float64})
 MethodInstance for func(::Vector{Int64}, ::Vector{Float64})

### Compilation pipeline

<p><img src="./imgs/from_source_to_native.png" alt="drawing" width="800"/></p>


### What makes Julia fast?

(Successful) **Type inference** -> **Specialization** -> **Compilation**

## Introspection tools
#### (*But I really want to see what happens!*)

We can inspect the code at all transformation stages with a bunch of macros:

<img src="./imgs/julia_introspection_macros.png" width=350px>

In [36]:
@macroexpand @time 3+3

quote
    #= timing.jl:263 =#
    begin
        #= timing.jl:268 =#
        $(Expr(:meta, :force_compile))
        #= timing.jl:269 =#
        local var"#202#stats" = Base.gc_num()
        #= timing.jl:270 =#
        local var"#204#elapsedtime" = Base.time_ns()
        #= timing.jl:271 =#
        Base.cumulative_compile_timing(true)
        #= timing.jl:272 =#
        local var"#205#compile_elapsedtimes" = Base.cumulative_compile_time_ns()
        #= timing.jl:273 =#
        local var"#203#val" = $(Expr(:tryfinally, :(3 + 3), quote
    var"#204#elapsedtime" = Base.time_ns() - var"#204#elapsedtime"
    #= timing.jl:275 =#
    Base.cumulative_compile_timing(false)
    #= timing.jl:276 =#
    var"#205#compile_elapsedtimes" = Base.cumulative_compile_time_ns() .- var"#205#compile_elapsedtimes"
end))
        #= timing.jl:278 =#
        local var"#206#diff" = Base.GC_Diff(Base.gc_num(), var"#202#stats")
        #= timing.jl:279 =#
        local var"#207#_msg" = Base.nothing
        #= timing.

In [37]:
@code_typed func(1.0,2.0)

CodeInfo(
1 ─ %1 = Base.mul_float(2.0, x)::Float64
│   %2 = Base.add_float(%1, y)::Float64
└──      return %2
) => Float64

From the types of the input arguments, Julia has figured out all the intermediate types and replaced the generic functions `*` and `+` by specific implementations (**static dispatch**). This crucial process is known as **type inference** and its success is the basis for a good specialization (i.e. performant native code as a result). It will concern us in much more detail tomorrow.

In [38]:
@code_llvm debuginfo=:none func(1.0,2.0)

define double @julia_func_2409(double %0, double %1) #0 {
top:
  %2 = fmul double %0, 2.000000e+00
  %3 = fadd double %2, %1
  ret double %3
}


In [39]:
@code_native debuginfo=:none func(1.0,2.0)

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 14, 0
	.globl	_julia_func_2425                ## -- Begin function julia_func_2425
	.p2align	4, 0x90
_julia_func_2425:                       ## @julia_func_2425
	.cfi_startproc
## %bb.0:                               ## %top
	vaddsd	%xmm0, %xmm0, %xmm0
	vaddsd	%xmm1, %xmm0, %xmm0
	retq
	.cfi_endproc
                                        ## -- End function
.subsections_via_symbols


Let's compare this to integer input.


In [40]:
@code_native debuginfo=:none func(1,2)

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 14, 0
	.globl	_julia_func_2427                ## -- Begin function julia_func_2427
	.p2align	4, 0x90
_julia_func_2427:                       ## @julia_func_2427
	.cfi_startproc
## %bb.0:                               ## %top
	leaq	(%rsi,%rdi,2), %rax
	retq
	.cfi_endproc
                                        ## -- End function
.subsections_via_symbols


### Recommendation: [Cthulhu.jl](https://github.com/JuliaDebug/Cthulhu.jl)
While these introspection macros are great, I recommend to use `@descend` from the package [Cthulhu.jl](https://github.com/JuliaDebug/Cthulhu.jl) for real world code analysis.

Essentially, Cthulhu is an **interactive**, more powerful generalization of the macros above.

* Allows easy switching between code representations (syntax, typed, native, ...).
* **Recursive application possible**(!) (i.e. introspecting a function that is called within a function within function ...).

However, due to its interactivity, it doesn't work in Jupyter but **only works in the REPL** (→ exercise).

<img src="imgs/cthulhu.png" width=1000>

## How important is specialization?

Let's try to estimate the performance gain by specialization.

To prevent specialization, we deliberately throw away any useful type information and operate on a `Vector{Any}` that can literally store anything!

(This is qualitatively comparable to what Python does.)


In [2]:
func(v) = 2*v[1] + v[2] # version of func that takes in a vector

func (generic function with 1 method)

In [53]:
rand(2)

2-element Vector{Float64}:
 0.7794739907620628
 0.20580422926731945

In [54]:
Any[rand(), rand()]

2-element Vector{Any}:
 0.42689498894071154
 0.5196898151534661

For benchmarking we will use `@btime` (or `@benchmark`) from [BenchmarkTools.jl](https://github.com/JuliaCI/BenchmarkTools.jl). This will take care of a couple of things for us:
* Exclude first run.
* Run the code multiple times (→ statistics).
* Benchmark in a function (local scope).

**General rule:** For proper benchmarking don't use `@time` but `@btime` and interpolate (`$`) global input arguments.

In [71]:
using BenchmarkTools

v_typed = rand(2)
v_any = Any[rand(), rand()]

@btime func($v_typed);
@btime func($v_any);

  3.911 ns (0 allocations: 0 bytes)
  59.210 ns (2 allocations: 32 bytes)


In [75]:
@benchmark func($v_any)

BenchmarkTools.Trial: 10000 samples with 982 evaluations.
 Range (min … max):  59.281 ns …  1.368 μs  ┊ GC (min … max): 0.00% … 93.44%
 Time  (median):     60.903 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   62.696 ns ± 31.323 ns  ┊ GC (mean ± σ):  1.18% ±  2.28%

   ██▃                                                         
  ▅███▇▆▆▄▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▂▁▁▂▁▂▂▂▂▂ ▃
  59.3 ns         Histogram: frequency by time        92.5 ns <

 Memory estimate: 32 bytes, allocs estimate: 2.

In [7]:
@code_typed func(rand(2))

CodeInfo(
1 ─ %1 = Base.arrayref(true, v, 1)::Float64
│   %2 = Base.mul_float(2.0, %1)::Float64
│   %3 = Base.arrayref(true, v, 2)::Float64
│   %4 = Base.add_float(%2, %3)::Float64
└──      return %4
) => Float64

In [8]:
@code_typed func(Any[rand(), rand()])

CodeInfo(
1 ─ %1 = Base.arrayref(true, v, 1)::Any
│   %2 = (2 * %1)::Any
│   %3 = Base.arrayref(true, v, 2)::Any
│   %4 = (%2 + %3)::Any
└──      return %4
) => Any

Note that in the latter case the generic functions `*` and `+` can not be replaced by specific variants due to lack of type information. This leads to inefficient **runtime dispatch**.

## Dispatch and specialization

**Types drive both dispatch and specialization.**

First, the most specific method is selected (dispatch), then it gets compiled to efficient native code (specialization).

In [7]:
myabs(x::Real) = sign(x) * x
myabs(z::Complex) = sqrt(real(z * conj(z)))

myabs (generic function with 2 methods)

In [22]:
@code_native myabs(3.2 + 4.5im)

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 14, 0
	.globl	_julia_myabs_1486               ## -- Begin function julia_myabs_1486
	.p2align	4, 0x90
_julia_myabs_1486:                      ## @julia_myabs_1486
; ┌ @ /Users/crstnbr/repos/JuliaWorkshops/JuliaHLRS23/notebooks/1_2_specialization.ipynb:2 within `myabs`
	.cfi_startproc
## %bb.0:                               ## %top
; │┌ @ complex.jl:293 within `*` @ float.jl:410
	vmovupd	(%rdi), %xmm0
	vmulpd	%xmm0, %xmm0, %xmm0
; ││ @ complex.jl:293 within `*`
; ││┌ @ float.jl:409 within `-`
	vpermilpd	$1, %xmm0, %xmm1        ## xmm1 = xmm0[1,0]
	vaddsd	%xmm1, %xmm0, %xmm0
; │└└
; │┌ @ math.jl:678 within `sqrt`
	vsqrtsd	%xmm0, %xmm0, %xmm0
; │└
	retq
	.cfi_endproc
; └
                                        ## -- End function
.subsections_via_symbols


In [23]:
@code_native myabs(3 + 4im)

	.section	__TEXT,__text,regular,pure_instructions
	.build_version macos, 14, 0
	.globl	_julia_myabs_1489               ## -- Begin function julia_myabs_1489
	.p2align	4, 0x90
_julia_myabs_1489:                      ## @julia_myabs_1489
; ┌ @ /Users/crstnbr/repos/JuliaWorkshops/JuliaHLRS23/notebooks/1_2_specialization.ipynb:2 within `myabs`
	.cfi_startproc
## %bb.0:                               ## %top
	subq	$8, %rsp
	.cfi_def_cfa_offset 16
; │┌ @ complex.jl:293 within `*` @ int.jl:88
	movq	(%rdi), %rax
; │└
; │┌ @ complex.jl:279 within `conj`
; ││┌ @ int.jl:85 within `-`
	movq	8(%rdi), %rcx
; │└└
; │┌ @ complex.jl:293 within `*` @ int.jl:88
	imulq	%rax, %rax
	imulq	%rcx, %rcx
; ││ @ complex.jl:293 within `*`
; ││┌ @ int.jl:86 within `-`
	addq	%rax, %rcx
; │└└
; │┌ @ math.jl:1489 within `sqrt`
; ││┌ @ float.jl:294 within `float`
; │││┌ @ float.jl:268 within `AbstractFloat`
; ││││┌ @ float.jl:159 within `Float64`
	vcvtsi2sd	%rcx, %xmm0, %xmm0
; ││└└└
; ││ @ math.jl:1491 within `sqrt` @ 

## Are explicit type annotations necessary? (think C or Fortran)

Note that Julia's type inference is powerful. Specifying types **is not** necessary for best performance!


In [ ]:
function my_function(x)
    y = rand()
    z = rand()
    x+y+z
end

function my_function_typed(x::Int)::Float64
    y::Float64 = rand()
    z::Float64 = rand()
    x+y+z
end

In [ ]:
@btime my_function(10);
@btime my_function_typed(10);

Annotating types explicitly can serve a purpose.

* Enforce conversions
* Very rarely: help the compiler infer types in tricky situations

However, more often than not it is an indication of suboptimal code design. (It also makes functions much less generic and reusable!)

## Note for heterogeneous HPC clusters

By default, Julia produces native code for the CPU type it is running on. This means that it uses the [Instruction Set Architecture (ISA)](https://en.wikipedia.org/wiki/Instruction_set_architecture) of this CPU.

This can lead to issues on heterogeneous clusters where different nodes have different CPU types. E.g. you precompile Julia packages on a login node with an Intel CPU but want to run the code on a compute node with AMD CPUs.

**Solution: Multiversioning**

```julia
export JULIA_CPU_TARGET="generic;zen2,clone_all;skylake,clone_all"
```

This will compile a generic (but slow) variant as well as efficient variants for AMD Zen2 and Intel Skylake CPUs.

# Core messages of this Notebook

* **A function is compiled when called for the first time** with a given set of argument types.
* The are **multiple code transformation steps** which can be inspected through macros like `@code_warntype` or `@descend` from Cthulhu.jl.
* What makes Julia fast? Successful **Type inference** → **Specialization** → **Compilation**.
* Functions should almost always be benchmarked with **BenchmarkTools.jl's `@btime` and `@benchmark`** instead of `@time`.
* In virtually all cases, **explicit type annotations are irrelevant for performance**.